In [1]:
#IMPORTING DEPENDENCIES

from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
# Scrape NASA Mars News
browser = init_browser()

# Visit NASA Mars News site
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

time.sleep(1)

# Instantiate BS object
html = browser.html
soup = bs(html, "html.parser")

# Find news slide
news_slide = soup.find('li', class_='slide')

# Scrape slide title
news_title = news_slide.find('div', class_='content_title').get_text()

# Scrape slide text
news_preview  = news_slide.find('div', class_='article_teaser_body').get_text()

# Store data in a dictionary
news_data = {
    
    "title": news_title,
    "content": news_preview
}

# Close the browser after scraping
browser.quit()

In [15]:
# Scrape JPL Mars Featured Images
browser = init_browser()

# Visit JPL site
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(3)

# Instantiate BS object
html = browser.html
soup = bs(html, "html.parser")

#browser.links.find_by_partial_text('FULL IMAGE').click()

# Find featured image link
img_url = soup('article', class_='carousel_item')[0]['style'][23:-3]
jpl_url = "https://www.jpl.nasa.gov" + img_url

# Close the browser after scraping
browser.quit()

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19113-1920x1200.jpg'